In [9]:
import os
import time
import pickle
import numpy as np
import pandas as pd
from docx import Document
from docx.shared import Pt
import docx2pdf

In [10]:
l=pd.concat([pd.read_csv('buyers.csv'),pd.read_csv('supplies.csv')], axis=0, ignore_index=True).to_dict('records')

with open('final_gstr3b_params.pkl', 'rb') as f:
    x1=pickle.load(f)
gstr3b={i[1]:i[3] for i in x1}

with open('final_gstr1_params.pkl', 'rb') as f:
    x2=pickle.load(f)
gstr1={i[1]:i[3] for i in x2}

In [12]:
import tqdm as tqdm

def update_document(row, col, new_value, table):
    if row >= len(table.rows) or col >= len(table.columns):
        print("Warning: The specified cell does not exist in the table.")
        return    
    cell = table.rows[row].cells[col]
    cell.text = str(new_value)
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            run.font.name, run.font.size = 'Roboto', Pt(7.5)

def process_document(fname, vals, f, m):
    doc = Document(fname)
    all_tables = []
    for table in doc.tables:
        rows = len(table.rows)
        cols = len(table.columns)
        data = np.empty((rows, cols), dtype=object)
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                data[i, j] = cell.text.strip()
        all_tables.append(data)

    for i, j in vals:
        update_document(i[0], i[1], j, doc.tables[i[2]])

    output_docx = f"{f}_Updated_Vals_{m}.docx"
    doc.save(output_docx)
    # docx2pdf.convert(output_docx, f"{f}_Updated_{m}.pdf")
    # os.remove(output_docx)

In [13]:
start_time=time.time()
for m in tqdm.tqdm(range(len(l))):
    x=l[m]
    d_={}
    d1_={}
    for gstr, d in [(gstr3b, d_), (gstr1, d1_)]:
        for i in x.keys():
            l_ = [k for k, val in gstr.items() if val == i]
            if len(l_) != 0:
                d.update(dict(zip(l_, tuple([x[i]]) * len(l_))))
        for i in set(gstr.keys()).difference(set(d.keys())):
            d[i] = '?'
    process_document('gstr3b.docx', [(k, v) for k, v in d_.items()], 'gstr3b', m)
    process_document('gstr1.docx', [(k, v) for k, v in d1_.items()], 'gstr1', m)
    
    # if m%10==0:
    #     print(f'Completed Batch {m}')
    # print(f"Time elapsed after iteration {m+1}: {time.time()-start_time}")
    # start_time=time.time()
end_time=time.time()
print(f"{(end_time-start_time)/60:.4f}minutes")

  1%|          | 4/432 [00:04<07:48,  1.10s/it]


KeyboardInterrupt: 

In [5]:
os.listdir()

['a.ipynb',
 'a1_final.ipynb',
 'app1.py',
 'app_Prototype.py',
 'buyers.csv',
 'final_gstr1_params.pkl',
 'final_gstr3b_params.pkl',
 'gstr1.docx',
 'gstr1_Updated_Vals_0.docx',
 'gstr1_Updated_Vals_1.docx',
 'gstr3b.docx',
 'gstr3b_Updated_Vals_0.docx',
 'gstr3b_Updated_Vals_1.docx',
 'supplies.csv']

In [6]:
import os
import win32com.client

def convert_docx_to_pdf(docx_file, pdf_file):
    # Get the full path of the input and output files
    docx_file = os.path.abspath(docx_file)
    pdf_file = os.path.abspath(pdf_file)

    # Create a new instance of the Word application
    word_app = win32com.client.Dispatch("Word.Application")

    # Open the docx file
    doc = word_app.Documents.Open(docx_file)

    # Save as PDF
    doc.SaveAs(pdf_file, FileFormat=17)

    # Close the document
    doc.Close()

    # Quit Word application
    word_app.Quit()

# Example usage
if __name__ == "__main__":
    pdf_file = "output.pdf"
    start_time=time.time()
    for i in ['gstr1_Updated_Vals_0.docx','gstr1_Updated_Vals_1.docx','gstr3b_Updated_Vals_0.docx','gstr3b_Updated_Vals_1.docx']:
        convert_docx_to_pdf(i, pdf_file)
    end_time=time.time()
    time_diff=(end_time-start_time)
    minutes, seconds = divmod(time_diff, 60)
    print(f"Time taken: {int(minutes):02d}:{seconds:.4f}")

Time taken: 00:27.6384


In [7]:
pdf_file = "output.pdf"
start_time=time.time()
for i in ['gstr1_Updated_Vals_0.docx','gstr1_Updated_Vals_1.docx','gstr3b_Updated_Vals_0.docx','gstr3b_Updated_Vals_1.docx']:
    docx2pdf.convert(i, pdf_file)
end_time=time.time()
time_diff=(end_time-start_time)
minutes, seconds = divmod(time_diff, 60)
print(f"Time taken: {int(minutes):02d}:{seconds:.4f}")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Time taken: 00:26.3335


In [12]:
import comtypes.client
import docx

pdf_path='output.pdf'
start_time=time.time()
for i in ['gstr1_Updated_Vals_0.docx','gstr1_Updated_Vals_1.docx','gstr3b_Updated_Vals_0.docx','gstr3b_Updated_Vals_1.docx']:
    word_path=i
    docx.Document(word_path)
    word=comtypes.client.CreateObject('Word.Application')
    docx_path = os.path. abspath(word_path)
    pdf_path = os.path.abspath(pdf_path)
    pdf_format=17
    word.Visible=False
    in_file=word.Documents.Open(docx_path)
    in_file.SaveAs(pdf_path,FileFormat=pdf_format)
    in_file.Close()
    word.Quit()
end_time=time.time()
time_diff=(end_time-start_time)
minutes, seconds = divmod(time_diff, 60)
print(f"Time taken: {int(minutes):02d}:{seconds:.4f}")

Time taken: 00:19.7568


In [ ]:
# from spire.doc import *
# from spire.doc.common import *

In [14]:
from spire.doc import *
from spire.doc.common import *
        
# Create a Document object
document = Document()
# Load a Word DOCX file
start_time=time.time()
for i in ['gstr1_Updated_Vals_0.docx','gstr1_Updated_Vals_1.docx','gstr3b_Updated_Vals_0.docx','gstr3b_Updated_Vals_1.docx']:
    document.LoadFromFile(i)
# Or load a Word DOC file
#document.LoadFromFile("Sample.doc")

# Save the file to a PDF file
    document.SaveToFile("output.pdf", FileFormat.PDF)
    document.Close()
end_time=time.time()
time_diff=(end_time-start_time)
minutes, seconds = divmod(time_diff, 60)
print(f"Time taken: {int(minutes):02d}:{seconds:.4f}")


Time taken: 00:4.8774


In [15]:
!pip install spire.doc

  Using cached Spire.Doc-12.4.0-py3-none-win_amd64.whl.metadata (14 kB)
  Using cached plum_dispatch-1.7.4-py3-none-any.whl.metadata (1.8 kB)
Using cached Spire.Doc-12.4.0-py3-none-win_amd64.whl (27.1 MB)
Using cached plum_dispatch-1.7.4-py3-none-any.whl (24 kB)


In [20]:
!pip install plum-dispatch==1.7.4

In [59]:
!pip install comtypes

   ---------------------------------------- 0.0/201.2 kB ? eta -:--:--
   -- ------------------------------------- 10.2/201.2 kB ? eta -:--:--
   ----- --------------------------------- 30.7/201.2 kB 435.7 kB/s eta 0:00:01
   -------------------------------------- - 194.6/201.2 kB 2.0 MB/s eta 0:00:01
   -------------------------------------- - 194.6/201.2 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 201.2/201.2 kB 1.2 MB/s eta 0:00:00
